In [20]:
import pandas as pd
from keras.models import Model
from keras.optimizers import SGD, RMSprop, Adam
from keras.layers import Conv1D, Concatenate,Input, Dense, Embedding, merge, LSTM, concatenate, Flatten, Dropout, Lambda, BatchNormalization
from keras.regularizers import l2

In [17]:
dataAll = pd.read_csv('/Users/ruben.arana/git/beat-bookies/data/combinedData.csv', low_memory=False)

## CREATE TEAM EMBEDDINGS

In [27]:
teams = dataAll.VISITOR_TEAM.unique()

In [28]:
# Create indexes for teams
teamId2idx = {o:i for i,o in enumerate(teams)}
teamId2idx

{'Atlanta': 9,
 'Boston': 26,
 'Brooklyn': 18,
 'Charlotte': 17,
 'Chicago': 8,
 'Cleveland': 21,
 'Dallas': 23,
 'Denver': 13,
 'Detroit': 14,
 'Golden State': 25,
 'Houston': 11,
 'Indiana': 1,
 'L.A. Clippers': 10,
 'L.A. Lakers': 4,
 'Memphis': 22,
 'Miami': 28,
 'Milwaukee': 3,
 'Minnesota': 15,
 'New Orleans': 0,
 'New York': 5,
 'Orlando': 27,
 'Philadelphia': 19,
 'Phoenix': 24,
 'Portland': 16,
 'Sacramento': 6,
 'San Antonio': 12,
 'Toronto': 7,
 'Utah': 20,
 'Washington': 2}

In [29]:
# Replace all team nambes for the indexes
dataAll.VISITOR_TEAM = dataAll.VISITOR_TEAM.apply(lambda x: teamId2idx[x])
dataAll.HOME_TEAM = dataAll.HOME_TEAM.apply(lambda x: teamId2idx[x])

In [30]:
# Initialize params
teams_number = dataAll.VISITOR_TEAM.unique().size
num_features = 50

In [31]:
# Create embeddings
home_in = Input(shape=(1,), dtype='int64', name='home_in')
u = Embedding(teams_number, num_features, input_length=1, W_regularizer=l2(1e-4))(home_in)
visitor_in = Input(shape=(1,), dtype='int64', name='visitor_in')
m = Embedding(teams_number, num_features, input_length=1, W_regularizer=l2(1e-4))(visitor_in)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(29, 50, input_length=1, embeddings_regularizer=<keras.reg...)`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(29, 50, input_length=1, embeddings_regularizer=<keras.reg...)`
  """


## CREATE TRAINING AND VALIDATION SETS

In [32]:
# Splitting all data randomly between training and validation
# msk = np.random.rand(len(dataAll)) < 0.8
# trn = dataAll[msk]
# val = dataAll[~msk]

# Splitting all data keeping the first ones as training set
n = dataAll.shape[0]
train_size = 0.8

trn = dataAll.iloc[:int(n * train_size)]
val = dataAll.iloc[int(n * train_size):]

## SIMPLE MODEL USING EMBEDDING DOT PRODUCT

In [33]:
# Creating embedding dot product model
x = merge([u, m], mode='dot')
x = Flatten()(x)
model = Model([home_in, visitor_in], x)
model.compile(Adam(0.001), loss='mse')

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  
/usr/local/lib/python3.6/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [35]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
home_in (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
visitor_in (InputLayer)         (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 1, 50)        1450        home_in[0][0]                    
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 1, 50)        1450        visitor_in[0][0]                 
__________________________________________________________________________________________________
merge_2 (M

In [36]:
# Training the model
model.fit([trn.VISITOR_TEAM, trn.HOME_TEAM], trn.HOME_POINTS + trn.VISITOR_POINTS, batch_size=64, epochs=200, 
          validation_data=([val.VISITOR_TEAM, val.HOME_TEAM], val.HOME_POINTS + val.VISITOR_POINTS))

Train on 960 samples, validate on 241 samples
Epoch 1/200
960/960 [==============================] - 0s 330us/step - loss: 39325.2630 - val_loss: 38528.7951
Epoch 2/200
960/960 [==============================] - 0s 26us/step - loss: 39323.7667 - val_loss: 38527.5574
Epoch 3/200
960/960 [==============================] - 0s 43us/step - loss: 39320.9880 - val_loss: 38523.8821
Epoch 4/200
960/960 [==============================] - 0s 41us/step - loss: 39314.7292 - val_loss: 38515.2350
Epoch 5/200
960/960 [==============================] - 0s 41us/step - loss: 39301.7674 - val_loss: 38497.7634
Epoch 6/200
960/960 [==============================] - 0s 35us/step - loss: 39278.2617 - val_loss: 38467.5755
Epoch 7/200
960/960 [==============================] - 0s 34us/step - loss: 39240.1451 - val_loss: 38421.2113
Epoch 8/200
960/960 [==============================] - 0s 53us/step - loss: 39184.3956 - val_loss: 38355.9340
Epoch 9/200
960/960 [==============================] - 0s 41us/step - los

960/960 [==============================] - 0s 28us/step - loss: 11824.6535 - val_loss: 11201.9527
Epoch 75/200
960/960 [==============================] - 0s 29us/step - loss: 11411.4204 - val_loss: 10799.9222
Epoch 76/200
960/960 [==============================] - 0s 47us/step - loss: 11003.8914 - val_loss: 10404.7816
Epoch 77/200
960/960 [==============================] - 0s 36us/step - loss: 10605.3621 - val_loss: 10018.7938
Epoch 78/200
960/960 [==============================] - 0s 34us/step - loss: 10215.4863 - val_loss: 9639.9675
Epoch 79/200
960/960 [==============================] - 0s 33us/step - loss: 9832.9430 - val_loss: 9270.3293
Epoch 80/200
960/960 [==============================] - 0s 30us/step - loss: 9460.1794 - val_loss: 8907.6559
Epoch 81/200
960/960 [==============================] - 0s 28us/step - loss: 9094.4761 - val_loss: 8556.2189
Epoch 82/200
960/960 [==============================] - 0s 35us/step - loss: 8739.2470 - val_loss: 8211.1158
Epoch 83/200
960/960 [=

Epoch 149/200
960/960 [==============================] - 0s 33us/step - loss: 413.2732 - val_loss: 384.0385
Epoch 150/200
960/960 [==============================] - 0s 51us/step - loss: 405.3238 - val_loss: 378.6240
Epoch 151/200
960/960 [==============================] - 0s 35us/step - loss: 398.1281 - val_loss: 373.5586
Epoch 152/200
960/960 [==============================] - 0s 49us/step - loss: 391.5359 - val_loss: 369.2505
Epoch 153/200
960/960 [==============================] - 0s 41us/step - loss: 385.6765 - val_loss: 364.9954
Epoch 154/200
960/960 [==============================] - 0s 50us/step - loss: 379.9188 - val_loss: 361.3913
Epoch 155/200
960/960 [==============================] - 0s 43us/step - loss: 374.8662 - val_loss: 358.2630
Epoch 156/200
960/960 [==============================] - 0s 33us/step - loss: 370.4182 - val_loss: 355.1995
Epoch 157/200
960/960 [==============================] - 0s 41us/step - loss: 366.1617 - val_loss: 352.4779
Epoch 158/200
960/960 [=====

In [ ]:
## CREATING A SIMPLE NEURAL NETWORK MODEL USING EMMBEDINGS

In [37]:
# Creating the neural network
x = merge([u, m], mode='concat')
x = Flatten()(x)
#x = Dropout(0.0)(x)
#x = Dense(50, activation='relu')(x)
#x = Dropout(0.0)(x)
x = Dense(1)(x)
neuralModel = Model([visitor_in, home_in], x)
neuralModel.compile(Adam(0.001), loss='mse')

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  
/usr/local/lib/python3.6/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [38]:
neuralModel.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
home_in (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
visitor_in (InputLayer)         (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 1, 50)        1450        home_in[0][0]                    
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 1, 50)        1450        visitor_in[0][0]                 
__________________________________________________________________________________________________
merge_3 (M

In [40]:
# Training the neural network
neuralModel.fit([trn.VISITOR_TEAM, trn.HOME_TEAM], trn.HOME_POINTS + trn.VISITOR_POINTS, batch_size=64, epochs=300, 
          validation_data=([val.VISITOR_TEAM, val.HOME_TEAM], val.HOME_POINTS + val.VISITOR_POINTS))

Train on 960 samples, validate on 241 samples
Epoch 1/300
960/960 [==============================] - 0s 34us/step - loss: 438.2673 - val_loss: 370.8574
Epoch 2/300
960/960 [==============================] - 0s 36us/step - loss: 427.2243 - val_loss: 363.4533
Epoch 3/300
960/960 [==============================] - 0s 79us/step - loss: 417.4089 - val_loss: 357.3821
Epoch 4/300
960/960 [==============================] - 0s 38us/step - loss: 409.0067 - val_loss: 352.1875
Epoch 5/300
960/960 [==============================] - 0s 51us/step - loss: 401.5692 - val_loss: 348.0520
Epoch 6/300
960/960 [==============================] - 0s 33us/step - loss: 395.3753 - val_loss: 344.4025
Epoch 7/300
960/960 [==============================] - 0s 33us/step - loss: 389.7479 - val_loss: 341.5195
Epoch 8/300
960/960 [==============================] - 0s 52us/step - loss: 384.7974 - val_loss: 339.1883
Epoch 9/300
960/960 [==============================] - 0s 35us/step - loss: 380.5352 - val_loss: 337.2635


960/960 [==============================] - 0s 26us/step - loss: 323.5759 - val_loss: 334.5582
Epoch 78/300
960/960 [==============================] - 0s 26us/step - loss: 323.3268 - val_loss: 334.7985
Epoch 79/300
960/960 [==============================] - 0s 27us/step - loss: 323.1291 - val_loss: 334.7764
Epoch 80/300
960/960 [==============================] - 0s 42us/step - loss: 322.9248 - val_loss: 334.9820
Epoch 81/300
960/960 [==============================] - 0s 31us/step - loss: 322.7458 - val_loss: 335.1448
Epoch 82/300
960/960 [==============================] - 0s 30us/step - loss: 322.5268 - val_loss: 335.2619
Epoch 83/300
960/960 [==============================] - 0s 33us/step - loss: 322.3001 - val_loss: 335.5580
Epoch 84/300
960/960 [==============================] - 0s 23us/step - loss: 322.2052 - val_loss: 335.4827
Epoch 85/300
960/960 [==============================] - 0s 27us/step - loss: 321.9899 - val_loss: 335.8127
Epoch 86/300
960/960 [============================

960/960 [==============================] - 0s 42us/step - loss: 318.0071 - val_loss: 344.4448
Epoch 154/300
960/960 [==============================] - 0s 34us/step - loss: 317.9622 - val_loss: 344.4334
Epoch 155/300
960/960 [==============================] - 0s 31us/step - loss: 317.9444 - val_loss: 344.4290
Epoch 156/300
960/960 [==============================] - 0s 52us/step - loss: 317.9025 - val_loss: 344.3292
Epoch 157/300
960/960 [==============================] - 0s 34us/step - loss: 317.9522 - val_loss: 344.3228
Epoch 158/300
960/960 [==============================] - 0s 57us/step - loss: 317.9210 - val_loss: 344.5314
Epoch 159/300
960/960 [==============================] - 0s 33us/step - loss: 317.9468 - val_loss: 344.4383
Epoch 160/300
960/960 [==============================] - 0s 59us/step - loss: 317.9257 - val_loss: 344.5323
Epoch 161/300
960/960 [==============================] - 0s 45us/step - loss: 317.9823 - val_loss: 344.5414
Epoch 162/300
960/960 [===================

Epoch 229/300
960/960 [==============================] - 0s 64us/step - loss: 317.8897 - val_loss: 346.3196
Epoch 230/300
960/960 [==============================] - 0s 32us/step - loss: 317.9295 - val_loss: 346.4416
Epoch 231/300
960/960 [==============================] - 0s 52us/step - loss: 317.8538 - val_loss: 346.1283
Epoch 232/300
960/960 [==============================] - 0s 38us/step - loss: 317.8966 - val_loss: 346.7230
Epoch 233/300
960/960 [==============================] - 0s 45us/step - loss: 317.8720 - val_loss: 346.3502
Epoch 234/300
960/960 [==============================] - 0s 51us/step - loss: 317.8892 - val_loss: 346.3580
Epoch 235/300
960/960 [==============================] - 0s 57us/step - loss: 317.8565 - val_loss: 346.1373
Epoch 236/300
960/960 [==============================] - 0s 41us/step - loss: 317.9593 - val_loss: 346.0998
Epoch 237/300
960/960 [==============================] - 0s 49us/step - loss: 317.8714 - val_loss: 346.3206
Epoch 238/300
960/960 [=====